In [8]:
import pandas as pd
import numpy as np
import os
import time
import utils
import pyodbc
import subprocess

In [9]:
# 将可转债features转移到raw文件中
def convert_features(feature):
    for csvname in os.listdir('day1_data_raw'):
        features = pd.read_csv(f'STKInfo/Bond/{csvname}', parse_dates = ['TRADINGDATE'])
        features.set_index('TRADINGDATE', inplace = True)
        data = pd.read_csv(f'day1_data_raw/{csvname}', parse_dates = ['date'])
        data.set_index('date', inplace = True)
        data[feature] = features[feature]
        data.to_csv(f'day1_data_raw/{csvname}')

In [10]:
convert_features('CONVERTIBLEPREMIUMRATE')

In [11]:
# 将可转债市值转移到raw文件中
df = pd.read_csv('STKInfo/BOND_ConvertInfo.csv')
for csvname in os.listdir('day1_data_raw'):
    stock = csvname[:9]
    market_cap = (df[df['SYMBOL9'] == stock]['ISSUEQUANTITY'] * 100000000/100).iloc[0]
    data = pd.read_csv(f'day1_data_raw/{csvname}', parse_dates = ['date'])
    data.set_index('date', inplace = True)
    data['marketcap'] = data['close'] * market_cap
    data.to_csv(f'day1_data_raw/{csvname}')


In [12]:
# 将新债的标记转移到raw中
newbond = pd.read_csv('STKInfo/Array/Fund_NewBnd.csv', parse_dates=['TRADINGDATE'])
newbond = newbond.pivot(index = 'TRADINGDATE', columns = 'SYMBOL9', values = 'FACTORVALUE')
for csvname in os.listdir('day1_data_raw'):
    stock = csvname[:9]
    data = pd.read_csv(f'day1_data_raw/{csvname}', parse_dates = ['date'])
    data.set_index('date', inplace = True)
    try:
        data['new'] = newbond.loc[:,stock]
    except:
        data['new'] = np.nan
    data.to_csv(f'day1_data_raw/{csvname}')

In [13]:
# 将正股ST的标记移到raw中
ST = pd.read_csv('STKInfo/Array/Fund_FlagST.csv', parse_dates = ['TRADINGDATE'])
ST = ST.pivot(index = 'TRADINGDATE', columns = 'SYMBOL9', values = 'FACTORVALUE')
for csvname in os.listdir('day1_data_raw'):
    stock = csvname[:9]
    data = pd.read_csv(f'day1_data_raw/{csvname}', parse_dates = ['date'])
    data.set_index('date', inplace = True)
    try:
        data['ST'] = ST.loc[:,stock]
    except:
        data['ST'] = np.nan
    data.to_csv(f'day1_data_raw/{csvname}')

In [14]:
'''
# 将原始数据转换成中间数据（csv）
'''
features = ['datetime','instrument','open','high','low','close','volume','pre_close','new','ST','CONVERTIBLEPREMIUMRATE','marketcap']
for csvname in os.listdir('day1_data_raw'):
    instrument = csvname[:9]
    print(f"正在转换中间文件：{instrument}...")
    df_tmp = pd.read_csv(f'day1_data_raw/{instrument}.csv')
    df_tmp['datetime'] = pd.to_datetime(df_tmp['date'], format='%Y-%m-%d')
    df_tmp['instrument'] = instrument
    df_tmp = df_tmp[features]
    df_tmp.set_index(['datetime', 'instrument'], inplace=True)
    df_tmp.to_csv(f'day1_data_mid/{instrument}.csv')

正在转换中间文件：110034.SH...
正在转换中间文件：110038.SH...
正在转换中间文件：110043.SH...
正在转换中间文件：110044.SH...
正在转换中间文件：110045.SH...
正在转换中间文件：110047.SH...
正在转换中间文件：110048.SH...
正在转换中间文件：110052.SH...
正在转换中间文件：110053.SH...
正在转换中间文件：110055.SH...
正在转换中间文件：110056.SH...
正在转换中间文件：110057.SH...
正在转换中间文件：110058.SH...
正在转换中间文件：110059.SH...
正在转换中间文件：110060.SH...
正在转换中间文件：110061.SH...
正在转换中间文件：110062.SH...
正在转换中间文件：110063.SH...
正在转换中间文件：110064.SH...
正在转换中间文件：110066.SH...
正在转换中间文件：110067.SH...
正在转换中间文件：110068.SH...
正在转换中间文件：110070.SH...
正在转换中间文件：110071.SH...
正在转换中间文件：110072.SH...
正在转换中间文件：110073.SH...
正在转换中间文件：110074.SH...
正在转换中间文件：110075.SH...
正在转换中间文件：110076.SH...
正在转换中间文件：110077.SH...
正在转换中间文件：110079.SH...
正在转换中间文件：110080.SH...
正在转换中间文件：110081.SH...
正在转换中间文件：110082.SH...
正在转换中间文件：110083.SH...
正在转换中间文件：110084.SH...
正在转换中间文件：110085.SH...
正在转换中间文件：110086.SH...
正在转换中间文件：110087.SH...
正在转换中间文件：110088.SH...
正在转换中间文件：110089.SH...
正在转换中间文件：110090.SH...
正在转换中间文件：110091.SH...
正在转换中间文件：110092.SH...
正在转换中间文件：110093.SH...
正在转换中间文件：1

In [15]:
'''
csv转bin
其中
E:/PY_qlib_env/.venv/Scripts/python.exe 指python路径
E:/PY_qlib_env/.venv/Lib/site-packages/qlib/scripts/dump_bin.py dump_all 指qlib的dump_bin.py路径
--csv_path 原始数据路径
--qlib_dir 转换后数据路径
--freq 1min 转换后数据频率
--date_field_name datetime 转换后数据时间字段
--symbol_field_name instrument 转换后数据股票代码字段
--include_fields open,high,low,close,volume 转换后数据包含字段
'''
print(f"正在执行把csv文件转换为bin文件...")
subprocess.run(f'D:/anaconda/envs/py10/python.exe C:/Users/tantra/Desktop/data/.venv/Lib/site-packages/qlib/scripts/dump_bin.py dump_all --csv_path day1_data_mid --qlib_dir day1_data_qlib --freq day --date_field_name datetime --symbol_field_name instrument --include_fields open,high,low,close,volume,pre_close,new,ST,CONVERTIBLEPREMIUMRATE,marketcap')

正在执行把csv文件转换为bin文件...


CompletedProcess(args='D:/anaconda/envs/py10/python.exe C:/Users/tantra/Desktop/data/.venv/Lib/site-packages/qlib/scripts/dump_bin.py dump_all --csv_path day1_data_mid --qlib_dir day1_data_qlib --freq day --date_field_name datetime --symbol_field_name instrument --include_fields open,high,low,close,volume,pre_close,new,ST,CONVERTIBLEPREMIUMRATE,marketcap', returncode=0)